In [ ]:
pip install transformers


In [6]:
import pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Load Dataset
file_path = "/content/news_data_cleaned.csv"  # Path to your uploaded file
data = pd.read_csv(file_path)

# Inspect column names
print(data.columns)

# Specify the column names
text_column = "Cleaned Article Text"  # Update based on your dataset
output_column = "Generated Summary_pegasus"   # New column for storing generated summaries

# Check for missing columns or create the output column if it doesn't exist
if output_column not in data.columns:
    data[output_column] = ""


Index(['Cleaned Title', 'Cleaned Article Text'], dtype='object')


In [3]:
import torch

In [7]:
# @title Default title text
# Load Pegasus Model and Tokenizer
model_name = "google/pegasus-xsum"  # You can also use other Pegasus variants
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
def generate_summary(text, model, tokenizer, max_length=128, min_length=30):
    inputs = tokenizer(
        text,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Generate summaries for each row
print("Generating summaries...")
for index, row in data.iterrows():
    article_text = row[text_column]
    try:
        summary = generate_summary(article_text, model, tokenizer)
        data.at[index, output_column] = summary
        print(f"Processed {index + 1}/{len(data)}")
    except Exception as e:
        print(f"Error processing row {index + 1}: {e}")
output_file = "/content/data_with_summaries.csv"  # Specify output path
data.to_csv(output_file, index=False)
print(f"Summarized data saved to {output_file}")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating summaries...
Processed 1/90
Processed 2/90
Processed 3/90
Processed 4/90
Processed 5/90
Processed 6/90
Processed 7/90
Processed 8/90
Processed 9/90
Processed 10/90
Processed 11/90
Processed 12/90
Processed 13/90
Processed 14/90
Processed 15/90
Processed 16/90
Processed 17/90
Processed 18/90
Processed 19/90
Processed 20/90
Processed 21/90
Processed 22/90
Processed 23/90
Processed 24/90
Processed 25/90
Processed 26/90
Processed 27/90
Processed 28/90
Processed 29/90
Processed 30/90
Processed 31/90
Processed 32/90
Processed 33/90
Processed 34/90
Processed 35/90
Processed 36/90
Processed 37/90
Processed 38/90
Processed 39/90
Processed 40/90
Processed 41/90
Processed 42/90
Processed 43/90
Processed 44/90
Processed 45/90
Processed 46/90
Processed 47/90
Processed 48/90
Processed 49/90
Processed 50/90
Processed 51/90
Processed 52/90
Processed 53/90
Processed 54/90
Processed 55/90
Processed 56/90
Processed 57/90
Processed 58/90
Processed 59/90
Processed 60/90
Processed 61/90
Processed

In [1]:
import pandas as pd

# Load the datasets
data = pd.read_csv("/content/summaries.csv")
df = pd.read_csv("/content/updated_rouge_scores (1).csv")

# Concatenate the 'summary_bart' column from 'data' with 'df'
if 'summary_bart' in data.columns:
    df = pd.concat([df, data['Generated Summary_pegasus']], axis=1)
    print("Data concatenated successfully!")
else:
    raise ValueError("Column 'summary_bart' not found in the dataset 'summaries.csv'.")

# Save the updated DataFrame
df.to_csv("/content/updated_rouge_scores (1).csv", index=False)
print("Updated DataFrame saved to '/content/updated_rouge_scores.csv'.")

Data concatenated successfully!
Updated DataFrame saved to '/content/updated_rouge_scores.csv'.


In [3]:
pip install rouge-score nltk pandas

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1890a4f5d9f12ba9c6d3ee71575dc8210e0bfea56a30b2a0964e48e0ec5da31e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [4]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the dataset
reference_file_path = "/content/updated_rouge_scores (1).csv"  # Replace with your file path
data = pd.read_csv(reference_file_path)


# Define the reference column and T5-generated summary columns
reference_column = "Summary"  # Reference summaries
generated_columns = ["Generated Summary_pegasus"]

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    """
    Calculate ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    if pd.isnull(reference) or pd.isnull(generated):
        return 0.0, 0.0, 0.0  # Handle missing values gracefully

    scores = scorer.score(reference, generated)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure

    return rouge1, rouge2, rougeL

# Process each T5 model column
for gen_col in generated_columns:
    if gen_col in data.columns:
        print(f"Calculating ROUGE scores for {gen_col}")
        scores = data.apply(lambda row: calculate_rouge(row[reference_column], row[gen_col]), axis=1)

        # Add new columns for each ROUGE score
        data[f'ROUGE-1_{gen_col}'] = scores.apply(lambda x: x[0])
        data[f'ROUGE-2_{gen_col}'] = scores.apply(lambda x: x[1])
        data[f'ROUGE-L_{gen_col}'] = scores.apply(lambda x: x[2])
    else:
        print(f"Column {gen_col} not found in the dataset.")

# Save the results to a new CSV file
output_file_path = "/content/updated_rouge_scores (1).csv"
data.to_csv(output_file_path, index=False)

print(f"ROUGE scores saved to: {output_file_path}")

Calculating ROUGE scores for Generated Summary_pegasus
ROUGE scores saved to: /content/updated_rouge_scores (1).csv


In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Save the model
model_save_path = "pegasus_xsum_model"
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model and tokenizer saved to pegasus_xsum_model
